In [1]:
import json
import torch
import pickle
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, EarlyStoppingCallback
from torch.utils.data import Dataset
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.utils.class_weight import compute_class_weight
from pyvi import ViTokenizer

2025-04-08 14:33:14.222595: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-08 14:33:14.231289: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744122794.240277  944078 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744122794.243151  944078 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1744122794.251125  944078 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [2]:
# 1. Đọc file intents.json
with open("/home/asd/testModel/MentalModel/Dataset/Chatbot.json", "r", encoding="utf-8") as f:
# with open("/home/asd/testModel/MentalModel/Dataset/Chatbot_remake.json", "r", encoding="utf-8") as f:
    data = json.load(f)

In [3]:
# 2. Tạo danh sách văn bản và nhãn (với phân đoạn từ)
texts = []
labels = []
for intent in data["intents"]:
    for pattern in intent["patterns"]:
        # Phân đoạn từ tiếng Việt bằng pyvi
        segmented_text = ViTokenizer.tokenize(pattern)
        texts.append(segmented_text)
        labels.append(intent["tag"])

In [4]:
# 3. Mã hoá nhãn
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)
num_classes = len(set(labels_encoded))

In [5]:
# 4. Load tokenizer & model (DistilBERT)
model_name = "distilbert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(
    model_name, 
    num_labels=num_classes,
    problem_type="single_label_classification"
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
# 5. Định nghĩa Dataset
class IntentDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=64):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        
        encoding = self.tokenizer(
            text, 
            padding="max_length", 
            truncation=True, 
            max_length=self.max_length, 
            return_tensors="pt"
        )
        
        return {
            "input_ids": encoding["input_ids"].squeeze(0),
            "attention_mask": encoding["attention_mask"].squeeze(0),
            "labels": torch.tensor(label, dtype=torch.long)
        }

In [7]:
# 6. Định nghĩa hàm compute_metrics
def compute_metrics(pred):
    labels = pred.label_ids
    preds = np.argmax(pred.predictions, axis=1)
    accuracy = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(labels, preds, average='macro')
    return {
        "accuracy": accuracy,
        "precision_weighted": precision,
        "recall_weighted": recall,
        "f1_weighted": f1,
        "precision_macro": precision_macro,
        "recall_macro": recall_macro,
        "f1_macro": f1_macro
    }

In [8]:
# 7. Tính trọng số lớp
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(labels_encoded),
    y=labels_encoded
)
class_weights = torch.tensor(class_weights, dtype=torch.float).to('cuda' if torch.cuda.is_available() else 'cpu')

In [9]:
# 8. Custom Trainer
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs): 
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits")
        loss_fct = torch.nn.CrossEntropyLoss(weight=class_weights)
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        return (loss, outputs) if return_outputs else loss

In [10]:
# 9. Cấu hình TrainingArguments
training_args = TrainingArguments(
    output_dir="./distilbert_intent_model",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=200,
    weight_decay=0.01,
    learning_rate=3e-5,
    warmup_steps=20,
    logging_dir="./logs",
    logging_steps=10,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="f1_macro",
    greater_is_better=True,
)

/home/asd/testModel/.conda/lib/python3.11/site-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [11]:
# 10. Thực hiện Cross-Validation 5 Folds
kf = KFold(n_splits=5, shuffle=True, random_state=42)
fold_results = []
best_f1_macro = -1  # Theo dõi f1_macro tốt nhất
best_model = None   # Lưu trữ mô hình tốt nhất
best_fold = -1      # Theo dõi fold tốt nhất

for fold, (train_idx, eval_idx) in enumerate(kf.split(texts)):
    print(f"\nTraining Fold {fold + 1}/5")
    
    # Chia dữ liệu cho fold hiện tại
    texts_train = [texts[i] for i in train_idx]
    labels_train = [labels_encoded[i] for i in train_idx]
    texts_eval = [texts[i] for i in eval_idx]
    labels_eval = [labels_encoded[i] for i in eval_idx]
    
    # Tạo dataset
    train_dataset = IntentDataset(texts_train, labels_train, tokenizer)
    eval_dataset = IntentDataset(texts_eval, labels_eval, tokenizer)
    
    # Khởi tạo model mới cho mỗi fold
    model = AutoModelForSequenceClassification.from_pretrained(
        model_name, 
        num_labels=num_classes,
        problem_type="single_label_classification"
    )
    
    # Khởi tạo trainer với EarlyStoppingCallback
    trainer = CustomTrainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        compute_metrics=compute_metrics,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=10)] 
    )
    
    # Huấn luyện
    trainer.train()
    
    # Đánh giá
    eval_results = trainer.evaluate()
    fold_results.append(eval_results)
    
    # In chi tiết từng measure cho fold hiện tại
    print(f"\nFold {fold + 1} Results:")
    print(f"Accuracy: {eval_results['eval_accuracy']:.4f}")
    print(f"Weighted Precision: {eval_results['eval_precision_weighted']:.4f}")
    print(f"Weighted Recall: {eval_results['eval_recall_weighted']:.4f}")
    print(f"Weighted F1: {eval_results['eval_f1_weighted']:.4f}")
    print(f"Macro Precision: {eval_results['eval_precision_macro']:.4f}")
    print(f"Macro Recall: {eval_results['eval_recall_macro']:.4f}")
    print(f"Macro F1: {eval_results['eval_f1_macro']:.4f}")
    
    # Kiểm tra và lưu mô hình tốt nhất
    current_f1_macro = eval_results["eval_f1_macro"]
    if current_f1_macro > best_f1_macro:
        best_f1_macro = current_f1_macro
        best_model = model  # Lưu mô hình tốt nhất
        best_fold = fold + 1

print(f"\nBest Fold: {best_fold} with F1 Macro: {best_f1_macro:.4f}")


Training Fold 1/5


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,Precision Weighted,Recall Weighted,F1 Weighted,Precision Macro,Recall Macro,F1 Macro
1,4.249100,4.236205,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,4.236600,4.214064,0.053191,0.004271,0.053191,0.007781,0.003586,0.060000,0.006628
3,4.085200,4.076545,0.042553,0.006966,0.042553,0.011496,0.008311,0.037736,0.013086
4,3.898300,3.897638,0.053191,0.017376,0.053191,0.022081,0.024294,0.062147,0.029284
5,3.640700,3.782869,0.085106,0.107650,0.085106,0.054344,0.048473,0.088218,0.035776
6,3.202500,3.633778,0.085106,0.082004,0.085106,0.061734,0.067090,0.091808,0.062406
7,3.087500,3.475327,0.148936,0.165284,0.148936,0.119326,0.096690,0.137778,0.087500
8,2.731500,3.396481,0.159574,0.183688,0.159574,0.136879,0.107778,0.124444,0.094259
9,2.323700,3.307589,0.170213,0.181796,0.170213,0.152950,0.105695,0.125269,0.098956
10,2.213700,3.171640,0.212766,0.217908,0.212766,0.184202,0.144722,0.185000,0.137057


/home/asd/testModel/.conda/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/asd/testModel/.conda/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/asd/testModel/.conda/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/asd

/home/asd/testModel/.conda/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/asd/testModel/.conda/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/asd/testModel/.conda/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/asd

...

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,Precision Weighted,Recall Weighted,F1 Weighted,Precision Macro,Recall Macro,F1 Macro
1,4.249100,4.255461,0.010753,0.000448,0.010753,0.000860,0.000801,0.019231,0.001538
2,4.191000,4.185708,0.021505,0.022103,0.021505,0.015469,0.010101,0.022727,0.007974
3,3.912100,4.061662,0.043011,0.025762,0.043011,0.028147,0.040607,0.067797,0.044367
4,3.635800,3.943701,0.086022,0.055479,0.086022,0.059123,0.057348,0.109195,0.065491
5,3.403800,3.830407,0.096774,0.100358,0.096774,0.082284,0.083333,0.109290,0.079547
6,3.285100,3.697120,0.086022,0.065233,0.086022,0.065950,0.068280,0.104839,0.073656
7,2.822800,3.614873,0.129032,0.194444,0.129032,0.131388,0.125000,0.118687,0.102814
8,2.652800,3.496958,0.129032,0.161649,0.129032,0.129032,0.120313,0.122396,0.106250
9,2.316600,3.432514,0.150538,0.170251,0.150538,0.150538,0.129487,0.129487,0.121795
10,2.149000,3.349542,0.150538,0.152330,0.150538,0.143369,0.121212,0.141414,0.123737


/home/asd/testModel/.conda/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/asd/testModel/.conda/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/asd/testModel/.conda/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/asd


Fold 5 Results:
Accuracy: 0.2903
Weighted Precision: 0.3297
Weighted Recall: 0.2903
Weighted F1: 0.2929
Macro Precision: 0.2292
Macro Recall: 0.2227
Macro F1: 0.2129

Best Fold: 2 with F1 Macro: 0.2585


/home/asd/testModel/.conda/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/asd/testModel/.conda/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/asd/testModel/.conda/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/asd

In [12]:
# 11. Lưu mô hình tốt nhất
if best_model is not None:
    best_model.save_pretrained("distilbert_intent_model_best")
    tokenizer.save_pretrained("distilbert_intent_tokenizer")
    with open("label_encoder.pkl", "wb") as handle:
        pickle.dump(label_encoder, handle, protocol=pickle.HIGHEST_PROTOCOL)
    print("🎉 Đã lưu mô hình tốt nhất từ 5 folds!")
else:
    print("⚠️ Không tìm thấy mô hình tốt nhất để lưu!")

🎉 Đã lưu mô hình tốt nhất từ 5 folds!


In [13]:
# 12. Tính trung bình kết quả từ 5 Folds
avg_results = {
    "accuracy": np.mean([r["eval_accuracy"] for r in fold_results]),
    "precision_weighted": np.mean([r["eval_precision_weighted"] for r in fold_results]),
    "recall_weighted": np.mean([r["eval_recall_weighted"] for r in fold_results]),
    "f1_weighted": np.mean([r["eval_f1_weighted"] for r in fold_results]),
    "precision_macro": np.mean([r["eval_precision_macro"] for r in fold_results]),
    "recall_macro": np.mean([r["eval_recall_macro"] for r in fold_results]),
    "f1_macro": np.mean([r["eval_f1_macro"] for r in fold_results])
}

print("\nAverage Results Across 5 Folds:")
print(f"Accuracy: {avg_results['accuracy']:.4f}")
print(f"Weighted Precision: {avg_results['precision_weighted']:.4f}")
print(f"Weighted Recall: {avg_results['recall_weighted']:.4f}")
print(f"Weighted F1: {avg_results['f1_weighted']:.4f}")
print(f"Macro Precision: {avg_results['precision_macro']:.4f}")
print(f"Macro Recall: {avg_results['recall_macro']:.4f}")
print(f"Macro F1: {avg_results['f1_macro']:.4f}")


Average Results Across 5 Folds:
Accuracy: 0.3190
Weighted Precision: 0.3492
Weighted Recall: 0.3190
Weighted F1: 0.3132
Macro Precision: 0.2492
Macro Recall: 0.2462
Macro F1: 0.2327
